# Interpolation of R vs T curves between values of magnetic field and input current
in order to convert MCE resistance data to temperature 

In [115]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
os.chdir(r'C:\Users\Pierre\Desktop\Postdoc\Technical_stuff\PPMS\PPMS_Heat_capacity\PPMS_Heat-capacity_TempCal\He3_TempCal')

In [45]:
calname = '2021-01-15_He3Puck698_multicurve_all_currents.dat'
# Read MFD data from csv file:
# Use last string in header as column header
# Read data from column mfd_idx+2 since, the 3 first columns are x, y, z data
# squeeze=True in order to make it a Series instead of a DataFrame, since there is only one column for each mfd_idx
cal = pd.read_csv(calname, skiprows=7, header=0)
cal

,Temperature (K),Current (code),Active Current Code,Field (Oe),ThRes (ohms),slope of -R vs T (ohms/K),slope of -R vs T (%/K),Point-to-Point R change (ohms),Point-to-Point slope change (ohms/K),Delta R in Overlap (ohms),Delta R in Overlap (%)
0,0.329838,11,11,0.000,9456.026900,63579.680250,672.372032,NaN,NaN,NaN,NaN
1,0.330498,11,11,0.000,9414.240139,63109.659495,670.363817,-41.786761,470.020754,NaN,NaN
2,0.331159,11,11,0.000,9372.679264,62643.327129,668.360939,-41.560874,466.332367,NaN,NaN
3,0.331821,11,11,0.000,9331.342916,62180.652602,666.363386,-41.336349,462.674526,NaN,NaN
4,0.332485,11,11,0.000,9290.229739,61721.605636,664.371144,-41.113177,459.046967,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
42589,346.646201,3,3,8000.156,45.914963,0.082464,0.179602,-0.057158,0.000290,NaN,NaN
42590,347.339494,3,3,8000.156,45.857891,0.082175,0.179196,-0.057072,0.000289,NaN,NaN
42591,348.034172,3,3,8000.156,45.800905,0.081888,0.178791,-0.056986,0.000288,NaN,NaN
42592,348.730241,3,3,8000.156,45.744006,0.081601,0.178387,-0.056900,0.000287,NaN,NaN


In [114]:
cal = cal.iloc[:,:5]
# del cal['Active Current Code']
for cname in cal.columns:
    cal.rename(columns={cname:cname.strip(' ')}, inplace=True)
cal.sort_values('Current (code)', inplace=True, ignore_index=True)
cal

,Temperature (K),Current (code),Field (Oe),ThRes (ohms),I (uA)
0,349.427701,3,8000.156,45.687192,45.45
1,143.049323,3,4000.068,80.720136,45.45
2,143.335422,3,4000.068,80.620202,45.45
3,143.622092,3,4000.068,80.520355,45.45
4,143.909337,3,4000.068,80.420596,45.45
...,...,...,...,...,...
42589,0.677257,11,140000.578,1857.195784,0.09
42590,0.678611,11,140000.578,1853.461889,0.09
42591,0.679968,11,140000.578,1849.739352,0.09
42592,0.765040,11,140000.578,1649.207512,0.09


In [111]:
codename = '../2021-01-19_DSPCurrentCodes.txt'
Icodes = pd.read_csv(codename, sep=' ', names=['code', 'current', 'unit'])
Icodes['I (uA)'] = np.where(Icodes['unit']=='nA', Icodes['current']*1e-3,
                            np.where(Icodes['unit']=='mA', Icodes['current']*1e3, 
                                     Icodes['current']))
Icodes.sort_values('code', ignore_index=True)

,code,current,unit,I (uA)
0,0,0.455,mA,455.00
1,1,0.227,mA,227.00
2,2,90.910,uA,90.91
3,3,45.450,uA,45.45
4,4,22.730,uA,22.73
5,5,9.090,uA,9.09
6,6,4.550,uA,4.55
7,7,2.270,uA,2.27
8,8,910.000,nA,0.91
9,9,450.000,nA,0.45


In [ ]:
dcal = {}
for code in np.unique(cal['Current (code)']):
    dcal[code] = np.where(cal['Current (code)']==code, Icodes['I (uA)'][Icodes['code']==code].item(), 0)
    
cal['I (uA)'] = np.concatenate([dcal[i][dcal[i]>0] for i in np.unique(cal['Current (code)'])])

In [123]:
cal[cal['Current (code)']==5]

,Temperature (K),Current (code),Field (Oe),ThRes (ohms),I (uA)
9768,28.205401,5,8000.156,196.632357,9.09
9769,18.614278,5,6000.113,240.318225,9.09
9770,18.651507,5,6000.113,240.092462,9.09
9771,18.688810,5,6000.113,239.866875,9.09
9772,18.726187,5,6000.113,239.641463,9.09
...,...,...,...,...,...
14845,33.426323,5,0.000,179.495246,9.09
14846,33.160244,5,0.000,180.201097,9.09
14847,33.359604,5,0.000,179.671538,9.09
14848,33.226565,5,0.000,180.024464,9.09


In [121]:
%matplotlib
fig = plt.figure()
ax = fig.gca(projection='3d')
surf = ax.plot_trisurf(cal['I (uA)'], cal['Temperature (K)'], cal['ThRes (ohms)'])

Using matplotlib backend: Qt5Agg
